In [1]:
# Download and extract the dataset from CMS

import requests # to download the dataset
import zipfile # to extract from archive
import shutil # to write the dataset to file
import os # rename file to something more type-able

data_dir = '../data/'

url = 'https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Information-on-Prescription-Drugs/Downloads/Part_D_All_Drugs_2015.zip'
response = requests.get(url, stream=True)
with open(data_dir + 'dataset.zip', 'wb') as ds_zipout:
    shutil.copyfileobj(response.raw, ds_zipout)

zip = zipfile.ZipFile(data_dir + 'dataset.zip', 'r')
ds_filename = zip.namelist()[0]
zip.extract(ds_filename, path=data_dir)

'../data/Medicare_Drug_Spending_PartD_All_Drugs_YTD_2015_12_06_2016.xlsx'

In [2]:
# Read in the raw data file and focus only on the sheet with the data in it
from openpyxl import load_workbook
workbook = load_workbook(data_dir + ds_filename)
data_sheet = workbook['Data']

In [3]:
# Extract XLSX data into a pandas DataFrame
import pandas as pd
import numpy as np
df = pd.DataFrame(data_sheet.values)
df.columns = list(df.iloc[3].values)
df.drop(df.index[0:4], inplace=True)
df.index = np.arange(1, len(df) + 1)

df

,Brand Name,Generic Name,"Claim Count, 2011","Total Spending, 2011","Beneficiary Count, 2011","Total Annual Spending Per User, 2011","Unit Count, 2011","Average Cost Per Unit (Weighted), 2011","Beneficiary Count No LIS, 2011","Average Beneficiary Cost Share No LIS, 2011",...,"Total Spending, 2015","Beneficiary Count, 2015","Total Annual Spending Per User, 2015","Unit Count, 2015","Average Cost Per Unit (Weighted), 2015","Beneficiary Count No LIS, 2015","Average Beneficiary Cost Share No LIS, 2015","Beneficiary Count LIS, 2015","Average Beneficiary Cost Share LIS, 2015","Annual Change in Average Cost Per Unit, 2015"
1,10 WASH,SULFACETAMIDE SODIUM,24,1569.19,16,98.0744,5170,0.303518,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1ST TIER UNIFINE PENTIPS,"PEN NEEDLE, DIABETIC",2472,57666.7,893,64.5764,293160,0.196766,422,42.3472,...,257896,5173,49.8542,1335132,0.193205,2878,27.3053,2295,3.98789,0.0222081
3,1ST TIER UNIFINE PENTIPS PLUS,"PEN NEEDLE, DIABETIC",None,None,None,None,None,None,None,None,...,73396.6,1470,49.9297,369886,0.19843,710,28.0719,760,3.51078,None
4,60PSE-400GFN-20DM,GUAIFENESIN/DM/PSEUDOEPHEDRINE,12,350.1,11,31.8273,497,0.704427,None,None,...,None,None,None,None,None,None,None,None,None,None
5,8-MOP,METHOXSALEN,11,9003.26,None,None,298,30.2123,None,None,...,None,None,None,None,None,None,None,None,None,None
6,A-B OTIC,ANTIPYRINE/BENZOCAINE,30,212.86,29,7.34,451,0.471973,None,None,...,None,None,None,None,None,None,None,None,None,None
7,ABACAVIR,ABACAVIR SULFATE,None,None,None,None,None,None,None,None,...,2.45788e+07,9319,2637.5,4424163,5.55559,2046,236.942,7273,6.14634,-0.137496
8,ABACAVIR-LAMIVUDINE-ZIDOVUDINE,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,None,None,None,None,None,None,None,None,...,1.66262e+07,1621,10256.8,759933,21.8785,377,945.154,1244,24.1874,-0.0228733
9,ABELCET,AMPHOTERICIN B LIPID COMPLEX,363,455566,97,4696.56,49027,9.29215,49,402.048,...,552848,102,5420.08,47600,11.6144,68,303.14,34,31.55,None
10,ABILIFY,ARIPIPRAZOLE,2447965,1469661103,360675,4074.75,7.74748e+07,19.2186,57408,466.026,...,1.57243e+09,322582,4874.51,4.7855e+07,33.1224,53024,519.777,269558,16.4978,0.139258


In [4]:
# Capture only the drug names (we'll need this later)
df_drugnames = df.iloc[:, :2]
df_drugnames.columns = [
    'drugname_brand',
    'drugname_generic'
]

In [5]:
# Serialize drug names to feather file for use in both Python and R
import feather
feather.write_dataframe(df_drugnames, data_dir + 'drugnames.feather')

In [6]:
# Separate column groups by year
cols_by_year = [
    { 'year': 2011, 'start': 2, 'end': 12 },
    { 'year': 2012, 'start': 12, 'end': 22 },
    { 'year': 2013, 'start': 22, 'end': 32 },
    { 'year': 2014, 'start': 32, 'end': 42 },
    { 'year': 2015, 'start': 42, 'end': 53 },
]

df_years = { cols['year']: df.iloc[:, cols['start']:cols['end']] for cols in cols_by_year }

In [7]:
# Remove 2015's extra column for "Annual Change in Average Cost Per Unit" (we can calculate it, anyhow)
df_years[2015] = df_years[2015].drop(df_years[2015].columns[-1], axis=1)

In [8]:
# Make columns easier to type and more generic w.r.t. year
generic_columns = [
    "claim_count",
    "total_spending",
    "user_count",
    "total_spending_per_user",
    "unit_count",
    "unit_cost_wavg",
    "user_count_non_lowincome",
    "out_of_pocket_avg_non_lowincome",
    "user_count_lowincome",
    "out_of_pocket_avg_lowincome"
]

for year in df_years:
    df_years[year].columns = generic_columns

In [9]:
# Cast all column data to appropriate numeric types

# Suppress SettingWithCopyWarnings because I think it's
# tripping on the fact that we have a dict of DataFrames
pd.options.mode.chained_assignment = None
for year in df_years:
    for col in df_years[year].columns:
        df_years[year].loc[:, col] = pd.to_numeric(df_years[year][col])
pd.options.mode.chained_assignment = 'warn'

In [10]:
# Serialize data for each year to feather file for use in both Python and R
for year in df_years:
    feather.write_dataframe(df_years[year], data_dir + 'spending-' + str(year) + '.feather')